In [59]:
import pandas as pd
import ast

In [60]:
# Load the datasets
indian_food_df = pd.read_csv("indian_food.csv")
good_ingredients_mood_df = pd.read_csv("ingredients_mood-good.csv")
bad_ingredients_mood_df = pd.read_csv("ingredients_mood-bad.csv")
allergen_df = pd.read_csv("allergen.csv")

# Fixing allergen_df
allergen_df['allergens'] = allergen_df['allergens'].apply(ast.literal_eval)

In [3]:
# Example to filter based on mood to get relevant ingredients
def get_dishes_based_on_mood(mood):
    # Make empty dataframes for positive and negative dishes
    positive_dishes = pd.DataFrame(columns=indian_food_df.columns)
    negative_dishes = pd.DataFrame(columns=indian_food_df.columns)

    # Get all ingredients that have a non NaN value in the respective mood column
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # positive_ingredients = positive_ingredients.iloc[:, 0].str.lower()
    # negative_ingredients = negative_ingredients.iloc[:, 0].str.lower()

    # Get dishes to recommend from the positive_ingredients df based on the mood
    for index, row in indian_food_df.iterrows():
        for ingredient in positive_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                positive_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])
        for ingredient in negative_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                negative_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])

    # Remove duplicate rows in both dataframes
    positive_dishes = positive_dishes.drop_duplicates()
    negative_dishes = negative_dishes.drop_duplicates()

    return positive_dishes, negative_dishes


In [91]:
# Given a dataframe of dishes, filter out dishes that contain ingredients correlated to the allergens
def filter_dishes_by_allergens(dishes, allergies, get_good_dishes=True):
    bad_ingredients = []

    for index, row in allergen_df.iterrows():
        if row['allergens'] != []:
            for allergen in row['allergens']:
                if allergen in allergies:
                    bad_ingredients.append(row['ingredients'])
                    continue

    # Drop duplicates from bad ingredients
    bad_ingredients = list(set(bad_ingredients))
    print(bad_ingredients)

    # Given the bad ingredients, filter the dishes to remove dishes that contain any of the bad ingredients
    filtered_dishes = pd.DataFrame(columns=dishes.columns)
    for index, row in dishes.iterrows():
        # Terrible implementation but it is what it is 
        common_ingredients = [i for i in bad_ingredients if i in row["ingredients"].split(", ")]
        if common_ingredients == [] and get_good_dishes:
            filtered_dishes = pd.concat([filtered_dishes, dishes[dishes["name"] == row["name"]]])
        elif not get_good_dishes:
            filtered_dishes = pd.concat([filtered_dishes, dishes[dishes["name"] == row["name"]]])
    
    return filtered_dishes

In [4]:
def recommend_dishes(mood, allergies, is_vegan, like_spicy):
    # Get relevant ingredients for the mood
    positive_dishes, negative_dishes = get_dishes_based_on_mood(mood)
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # print(f"before filter: {negative_dishes}")

    # Exclude based on user restrictions
    if is_vegan:
        positive_dishes = positive_dishes[positive_dishes["diet"] == "vegetarian"]
    # for allergen in allergies:
    #     positive_dishes = positive_dishes[~positive_dishes['ingredients'].str.contains(allergen, case=False)]
    if allergies:
        positive_dishes = filter_dishes_by_allergens(positive_dishes, allergies, get_good_dishes=True)
    if not like_spicy:
        positive_dishes = positive_dishes[~positive_dishes['flavor_profile'].str.contains("spicy", case=False)]

    if is_vegan:
        negative_dishes = negative_dishes[negative_dishes["diet"] == "non vegetarian"]
    # if allergies:
    #     allergen_mask = pd.Series(False, index=negative_dishes.index)
    #     for allergen in allergies:
    #         # This updates the mask to True for any row where the allergen is found
    #         allergen_mask |= negative_dishes['ingredients'].str.contains(allergen, case=False)
    #     negative_dishes = negative_dishes[allergen_mask]
    if allergies:
        negative_dishes = filter_dishes_by_allergens(negative_dishes, allergies, get_good_dishes=False)
    if not like_spicy:
        negative_dishes = negative_dishes[negative_dishes['flavor_profile'].str.contains("spicy", case=False)]

    final_recom = positive_dishes.sample(n=3)  # For simplicity, randomly pick three satisfying all conditions
    # final_recom = positive_dishes.head(3)

    # print(f"after filter: {negative_dishes}")
    # FIXME: Some checks here that are present in recommender.py are missing here
    negative_dishes_empty = negative_dishes.empty
    avoid_recom = pd.DataFrame()
    if not negative_dishes_empty:
        avoid_recom = negative_dishes.sample(n=3)  # For simplicity, randomly pick three satisfying all conditions

    # Check if final_recom is empty
    if final_recom.empty:
        print("Sorry, I couldn't find any dishes satisfying your preferences and restrictions.")
    else:
        print(f"Based on your mood and preferences, I recommend trying the following dishes: ")
        for index, row in final_recom.iterrows():
            region_string = ""
            state_string = ""
            flavor_string = ""
            if row['region'] != "-1":
                region_string = f" of the {row['region']} region"
            if row['state'] != "-1":
                state_string = f" from the {row['state']} state"
            # print(type(row['flavor_profile']))
            if row['flavor_profile'] != "-1":
                flavor_string = f" is a {row['flavor_profile']} dish and"
            print(f"{row['name']}{state_string}{region_string}, which is a {row['course']} dish.")
            print(f"{row['name']}{flavor_string} is {row['diet']}.")
            print(f"{row['name']} contains the ingredients {row['ingredients']}.")
            for ingredient in row['ingredients'].split(","):
                # print(ingredient)
                ingredient = ingredient.strip()
                # print(positive_ingredients)
                # FIXME: Check if ingredient is really missing from positive_ingredients["ingredients"] or it's just a difference in capitalization
                # FIXME: If it's the former, then why is it missing if we got our recommendation based on the ingredients?
                if ingredient in positive_ingredients["ingredients"].values:
                    # print(positive_ingredients.loc[positive_ingredients['ingredients'] == ingredient, "Y Reason"].iloc[0])
                    ingredient_reason = positive_ingredients.loc[positive_ingredients['ingredients'] == ingredient, "Y Reason"].iloc[0]
                    # print(ingredient_reason)
                    if isinstance(ingredient_reason, str):
                        print("  " + ingredient_reason)

    # Check if avoid_recom is empty
    if negative_dishes_empty:
        # print("I couldn't find any dishes that you should avoid.")
        pass
    else:
        print(f"Based on your mood and preferences, I recommend avoiding the following dishes: ")
        for index, row in avoid_recom.iterrows():
            region_string = ""
            state_string = ""
            flavor_string = ""
            if row['region'] != "-1":
                region_string = f" of the {row['region']} region"
            if row['state'] != "-1":
                state_string = f" from the {row['state']} state"
            # print(type(row['flavor_profile']))
            if row['flavor_profile'] != "-1":
                flavor_string = f" is a {row['flavor_profile']} dish and"
            print(f"{row['name']}{state_string}{region_string}, which is a {row['course']} dish.")
            print(f"{row['name']}{flavor_string} is {row['diet']}.")
            print(f"{row['name']} contains the ingredients {row['ingredients']}.")
            for ingredient in row['ingredients'].split(","):
                # print(ingredient)
                ingredient = ingredient.strip()
                # print(negative_ingredients)
                if ingredient in negative_ingredients["ingredients"].values:
                    # print(negative_ingredients.loc[negative_ingredients['ingredients'] == ingredient, "N Reason"].iloc[0])
                    ingredient_reason = negative_ingredients.loc[negative_ingredients['ingredients'] == ingredient, "X Reason"].iloc[0]
                    # print(ingredient_reason)
                    if isinstance(ingredient_reason, str):
                        print("  " + ingredient_reason)

In [6]:
positive_ingredients2 = good_ingredients_mood_df[good_ingredients_mood_df["Disgust"].notna()]
positive_ingredients2

,ingredients,Sad,Anger,Disgust,Fear,Joy,Neutral,Y Reason,Source,translation
8,potato,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
10,garlic,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
11,ginger,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
21,fish,y,NaN,y,NaN,y,NaN,Fish contains Omega-3 and is a part of the Med...,NaN,NaN
23,Potatoes,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
24,hot water,NaN,NaN,y,NaN,NaN,NaN,"When feeling sick, hot water helps relieve con...",https://www.healthline.com/nutrition/drinking-...,NaN
40,yogurt,y,y,y,y,NaN,NaN,Yogurt is rich in Tryptophan. Tryptophan can i...,NaN,NaN
72,Chicken thighs,y,NaN,y,NaN,NaN,NaN,Chicken is a part of the Mediterranean diet.Th...,NaN,NaN
77,garlic powder,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN
80,chicken,y,y,y,NaN,NaN,NaN,Chicken contains Tryptophan and Selenium and i...,NaN,NaN


In [7]:
isinstance(positive_ingredients2.loc[positive_ingredients2['ingredients'] == "fish", "Y Reason"].iloc[0], str)

True

In [8]:
indian_food_df.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [9]:
good_ingredients_mood_df.head()

,ingredients,Sad,Anger,Disgust,Fear,Joy,Neutral,Y Reason,Source,translation
0,elachi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardamom
1,vegetable oil,y,NaN,NaN,NaN,NaN,NaN,Vegetable oil contains vitamin E which has bee...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,NaN
2,ghee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jaggery,NaN,NaN,NaN,NaN,y,NaN,NaN,,NaN
4,Rice flour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
get_dishes_based_on_mood("Disgust")[0]

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
11,Lassi,"Yogurt, milk, nuts, sugar",vegetarian,5,5,sweet,dessert,Punjab,North
38,Bandar laddu,"Besan, jaggery, cardamom powder, ghee, cashews...",vegetarian,5,35,sweet,dessert,Andhra Pradesh,South
64,Maach Jhol,"Fish, potol, tomato, chillies, ginger, garlic",non vegetarian,10,40,spicy,main course,Assam,North East
...,...,...,...,...,...,...,...,...,...
239,Koldil Chicken,"Banana flower, chicken, green chili, mustard o...",non vegetarian,-1,-1,spicy,main course,Assam,North East
242,Masor Koni,"Fish roe, pumpkin flowers, mustard oil, turmer...",non vegetarian,-1,-1,spicy,main course,Assam,North East
243,Mishti Chholar Dal,"Chana dal, fresh coconut, ginger, cinnamon, ra...",vegetarian,10,30,sweet,main course,West Bengal,East
247,Prawn malai curry,"Coconut milk, prawns, garlic, turmeric, sugar",non vegetarian,15,50,spicy,main course,West Bengal,East


In [11]:
recommend_dishes("Disgust", ["milk", "yogurt"], True, False)

Based on your mood and preferences, I recommend trying the following dishes: 
Mishti Chholar Dal from the West Bengal state of the East region, which is a main course dish.
Mishti Chholar Dal is a sweet dish and is vegetarian.
Mishti Chholar Dal contains the ingredients Chana dal, fresh coconut, ginger, cinnamon, raisins.
  Raisins is a fruit which is a part of the Mediterranean diet. The Mediterranean diet is associated with lower levels of depression as it causes less inflammation, less oxidative stress and improved vascular function.
Bandar laddu from the Andhra Pradesh state of the South region, which is a dessert dish.
Bandar laddu is a sweet dish and is vegetarian.
Bandar laddu contains the ingredients Besan, jaggery, cardamom powder, ghee, cashews and raisins, jaggery syrup, sugar.
Hando Guri from the Assam state of the North East region, which is a dessert dish.
Hando Guri is a sweet dish and is vegetarian.
Hando Guri contains the ingredients Jaggery, raisins.
  Raisins is a fr

In [12]:
recommend_dishes("Disgust", [], True, False)

Based on your mood and preferences, I recommend trying the following dishes: 
Paniyaram from the Tamil Nadu state of the South region, which is a main course dish.
Paniyaram is vegetarian.
Paniyaram contains the ingredients Yogurt, ginger, curry leaves, baking soda, green chilli.
Thayir sadam from the Tamil Nadu state of the South region, which is a main course dish.
Thayir sadam is vegetarian.
Thayir sadam contains the ingredients Urad dal, curd, sesame oil, ginger, curry leaves, mustard seeds.
Gajar ka halwa from the Punjab state of the North region, which is a dessert dish.
Gajar ka halwa is a sweet dish and is vegetarian.
Gajar ka halwa contains the ingredients Carrots, milk, sugar, ghee, cashews, raisins.
  Raisins is a fruit which is a part of the Mediterranean diet. The Mediterranean diet is associated with lower levels of depression as it causes less inflammation, less oxidative stress and improved vascular function.
Based on your mood and preferences, I recommend avoiding the 